#### using embedding 

In [23]:
import pandas as pd

In [24]:
# Read the CSV file
df = pd.read_csv('dataset.csv')

df.head()

,id,text,lemma,upos,xpos,head,deprel,start_char,end_char
0,1,اس,یہ,DET,DEM,2,det,0,2
1,2,سلسلے,سلسلہ,NOUN,NN,5,nmod,3,8
2,3,کی,کا,ADP,PSP,2,case,9,11
3,4,دیگر,دیگر,ADJ,JJ,5,amod,12,16
4,5,اقساط,اقساط,NOUN,NN,7,nsubj,17,22


In [25]:
data = df.drop(columns=['id'])

In [26]:
# Initialize a new column 'y' with the default value 'S_M'
data['y'] = 'S_M'

# Iterate through the rows to assign 'S_B'
for i in range(len(data) - 1):
    # Check if the current word ends with a full stop
    if data.loc[i, 'text'].endswith('۔'):
        # Assign 'S_B' to the next word
        if i + 1 < len(data):
            data.loc[i + 1, 'y'] = 'S_B'  # Sentence Beginning

# Convert 'y' column to categorical type (optional, for ML efficiency)
data['y'] = data['y'].astype('category')

# Map categorical labels to numeric values
label_mapping = {'S_B': 1, 'S_M': 0}
data['y'] = data['y'].map(label_mapping)

# Verify the result
data.head(10)

,text,lemma,upos,xpos,head,deprel,start_char,end_char,y
0,اس,یہ,DET,DEM,2,det,0,2,0
1,سلسلے,سلسلہ,NOUN,NN,5,nmod,3,8,0
2,کی,کا,ADP,PSP,2,case,9,11,0
3,دیگر,دیگر,ADJ,JJ,5,amod,12,16,0
4,اقساط,اقساط,NOUN,NN,7,nsubj,17,22,0
5,یہاں,یہاں,PRON,PRP,7,obl,23,27,0
6,پڑھیے,پڑھ,VERB,VM,0,root,28,33,0
7,۔,۔,PUNCT,SYM,7,punct,33,34,0
8,یہ,یہ,PRON,PRP,3,nsubj,36,38,1
9,کیسے,کیسا,PRON,WQ,3,advmod,39,43,0


In [27]:
# Drop rows where the 'text' column contains only punctuation
data = data[~data['text'].str.contains(r'^[^\w\s]+$', na=False)]

# Verify the result
data.head(10)

,text,lemma,upos,xpos,head,deprel,start_char,end_char,y
0,اس,یہ,DET,DEM,2,det,0,2,0
1,سلسلے,سلسلہ,NOUN,NN,5,nmod,3,8,0
2,کی,کا,ADP,PSP,2,case,9,11,0
3,دیگر,دیگر,ADJ,JJ,5,amod,12,16,0
4,اقساط,اقساط,NOUN,NN,7,nsubj,17,22,0
5,یہاں,یہاں,PRON,PRP,7,obl,23,27,0
6,پڑھیے,پڑھ,VERB,VM,0,root,28,33,0
8,یہ,یہ,PRON,PRP,3,nsubj,36,38,1
9,کیسے,کیسا,PRON,WQ,3,advmod,39,43,0
10,ممکن,ممکن,ADJ,JJ,0,root,44,48,0


In [28]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [29]:
# One-hot encode 'upos', 'xpos', and 'deprel'
encoder = OneHotEncoder(sparse_output=False)
encoded_cats = encoder.fit_transform(data[['upos', 'xpos', 'deprel']])

# Convert to DataFrame for easier merging
encoded_cats_df = pd.DataFrame(encoded_cats, columns=encoder.get_feature_names_out())

# Concatenate encoded features back to the dataset
data = pd.concat([data.reset_index(drop=True), encoded_cats_df], axis=1)

# Drop the original categorical columns (optional)
data = data.drop(columns=['upos', 'xpos', 'deprel','lemma'])

In [30]:
# Select the numerical features to normalize
numerical_features = ['start_char', 'end_char', 'head']

from sklearn.preprocessing import MinMaxScaler

# Option 2: Min-Max Scaling (scales features to a range, typically 0 to 1)
min_max_scaler = MinMaxScaler()
data[numerical_features] = min_max_scaler.fit_transform(data[numerical_features])

In [31]:
# Verify the result
data.head(10)

,text,head,start_char,end_char,y,upos_ADJ,upos_ADP,upos_ADV,upos_AUX,upos_CCONJ,...,deprel_mark,deprel_nmod,deprel_nsubj,deprel_nummod,deprel_obj,deprel_obl,deprel_punct,deprel_root,deprel_vocative,deprel_xcomp
0,اس,0.014493,0.000000,0.000000,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,سلسلے,0.036232,0.000003,0.000005,0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,کی,0.014493,0.000008,0.000008,0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,دیگر,0.036232,0.000011,0.000012,0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,اقساط,0.050725,0.000015,0.000018,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,یہاں,0.050725,0.000020,0.000022,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,پڑھیے,0.000000,0.000025,0.000027,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,یہ,0.021739,0.000032,0.000032,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,کیسے,0.021739,0.000035,0.000036,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,ممکن,0.000000,0.000039,0.000041,0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [11]:
from gensim.models import FastText, KeyedVectors
import numpy as np

In [13]:
from gensim.models import FastText

fasttext_model = FastText.load_fasttext_format("C:\\Users\\Kanza Nasim\\Downloads\\embeddings\\cc.ur.300.bin")





C:\Users\Kanza Nasim\AppData\Local\Temp\ipykernel_26552\3350117351.py:3: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  fasttext_model = FastText.load_fasttext_format("C:\\Users\\Kanza Nasim\\Downloads\\embeddings\\cc.ur.300.bin")


************************************************************* NO NO**********************************************************************

In [32]:
def get_sentence_embedding(sentence, model):
    """
    Computes the mean FastText embedding for a sentence.
    """
    words = sentence.split()
    embeddings = []
    for word in words:
        if word in model.wv:  # Check if the word is in the vocabulary
            embeddings.append(model.wv[word])  # Access the word vector using the 'wv' attribute
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model.vector_size)  # Return a zero vector if no words are found


# Generate FastText embeddings for each row
data['fasttext_embedding'] = data['text'].apply(lambda x: get_sentence_embedding(x, fasttext_model))

# Convert the embeddings into a numpy array
embedding_matrix = np.vstack(data['fasttext_embedding'].values)

# Drop the original text-related columns (optional)
data = data.drop(columns=['text', 'fasttext_embedding'])



In [33]:
# Add FastText embeddings as features
embedding_df = pd.DataFrame(embedding_matrix, columns=[f'ft_dim_{i}' for i in range(embedding_matrix.shape[1])])
data = pd.concat([data.reset_index(drop=True), embedding_df.reset_index(drop=True)], axis=1)

In [34]:
data.head()

,head,start_char,end_char,y,upos_ADJ,upos_ADP,upos_ADV,upos_AUX,upos_CCONJ,upos_DET,...,ft_dim_290,ft_dim_291,ft_dim_292,ft_dim_293,ft_dim_294,ft_dim_295,ft_dim_296,ft_dim_297,ft_dim_298,ft_dim_299
0,0.014493,0.000000,0.000000,0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.083593,-0.094064,-0.041935,-0.013302,-0.014009,-0.106306,0.029570,0.141715,-0.008373,-0.046092
1,0.036232,0.000003,0.000005,0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.009919,-0.005164,0.005828,-0.032672,-0.028227,0.011800,-0.019054,0.029461,0.015847,-0.001126
2,0.014493,0.000008,0.000008,0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.009689,-0.096879,-0.015673,0.040102,-0.020523,-0.046326,-0.042110,-0.048978,-0.052121,-0.037590
3,0.036232,0.000011,0.000012,0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.007919,-0.000579,0.012618,-0.003285,0.028999,0.023561,0.031377,-0.030129,-0.009216,0.040360
4,0.050725,0.000015,0.000018,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.090482,0.035621,-0.018918,-0.143578,-0.021579,0.056538,-0.083902,0.070562,-0.014618,-0.061094


In [35]:
# Define the feature matrix (drop 'y') and target
X = data.drop(columns=['y'])
y = data['y']

In [36]:
# Split into training (64%), validation (16%), and test (20%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.36, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.56, random_state=42, stratify=y_temp)

In [39]:

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
from sklearn import tree

# Initialize the Decision Tree Classifier
dt_model = DecisionTreeClassifier(random_state=42, max_depth=10)  
# Fit the model on the training set
dt_model.fit(X_train, y_train)

y_train_pred = dt_model.predict(X_train)
y_val_pred = dt_model.predict(X_val)
y_test_pred = dt_model.predict(X_test)

# Evaluate the model
print("Decision Tree - Train Set:")
print(classification_report(y_train, y_train_pred))
print("Decision Tree - Validation Set:")
print(classification_report(y_val, y_val_pred))

print("Decision Tree - Test Set:")
print(classification_report(y_test, y_test_pred))

Decision Tree - Train Set:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    148335
           1       0.82      0.64      0.72      5768

    accuracy                           0.98    154103
   macro avg       0.90      0.81      0.85    154103
weighted avg       0.98      0.98      0.98    154103

Decision Tree - Validation Set:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     36713
           1       0.78      0.57      0.66      1427

    accuracy                           0.98     38140
   macro avg       0.88      0.78      0.83     38140
weighted avg       0.98      0.98      0.98     38140

Decision Tree - Test Set:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     46727
           1       0.76      0.58      0.66      1817

    accuracy                           0.98     48544
   macro avg       0.87      0.79      0.82

In [40]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

In [41]:
# Initialize XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Fit the model
xgb_model.fit(X_train, y_train)

# Predict on validation and test sets
y_train_pred_xgb = xgb_model.predict(X_train)
y_val_pred_xgb = xgb_model.predict(X_val)
y_test_pred_xgb = xgb_model.predict(X_test)

# Evaluate XGBoost
print("XGBoost - Train Set:")
print(classification_report(y_train, y_train_pred_xgb))
print("XGBoost - Validation Set:")
print(classification_report(y_val, y_val_pred_xgb))
print("XGBoost - Test Set:")
print(classification_report(y_test, y_test_pred_xgb))

c:\Users\Kanza Nasim\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [13:13:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost - Train Set:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    148335
           1       0.87      0.74      0.80      5768

    accuracy                           0.99    154103
   macro avg       0.93      0.87      0.89    154103
weighted avg       0.99      0.99      0.99    154103

XGBoost - Validation Set:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     36713
           1       0.81      0.66      0.72      1427

    accuracy                           0.98     38140
   macro avg       0.90      0.82      0.86     38140
weighted avg       0.98      0.98      0.98     38140

XGBoost - Test Set:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     46727
           1       0.81      0.66      0.73      1817

    accuracy                           0.98     48544
   macro avg       0.90      0.83      0.86     48544
weighte

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
# Initialize Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model
rf_model.fit(X_train, y_train)

# Predict on validation and test sets
# also predict on the training set to see how well the model is doing
y_train_pred_rf = rf_model.predict(X_train)
y_val_pred_rf = rf_model.predict(X_val)
y_test_pred_rf = rf_model.predict(X_test)

# Evaluate Random Forest
print("Random Forest - Train Set:")
print(classification_report(y_train, y_train_pred_rf))
print("Random Forest - Validation Set:")
print(classification_report(y_val, y_val_pred_rf))
print("Random Forest - Test Set:")
print(classification_report(y_test, y_test_pred_rf))

Random Forest - Train Set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    148335
           1       1.00      1.00      1.00      5768

    accuracy                           1.00    154103
   macro avg       1.00      1.00      1.00    154103
weighted avg       1.00      1.00      1.00    154103

Random Forest - Validation Set:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     36713
           1       0.75      0.61      0.67      1427

    accuracy                           0.98     38140
   macro avg       0.87      0.80      0.83     38140
weighted avg       0.98      0.98      0.98     38140

Random Forest - Test Set:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     46727
           1       0.73      0.60      0.66      1817

    accuracy                           0.98     48544
   macro avg       0.86      0.80      0.82

In [54]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, classification_report

# Define a simple DNN model
def create_dnn_model(input_dim, num_classes):
    model = keras.Sequential([
        layers.InputLayer(input_shape=(input_dim,)),  # Corrected input layer definition
        layers.Dense(128, activation='relu'),  # Hidden layer with ReLU activation
        layers.Dropout(0.2),  # Dropout for regularization
        layers.Dense(64, activation='relu'),  # Another hidden layer
        layers.Dropout(0.2),  # Dropout for regularization
        layers.Dense(num_classes, activation='softmax')  # Output layer with softmax for multi-class classification
    ])
    
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy',  # Use sparse categorical crossentropy for integer labels
                  metrics=['accuracy'])
    
    return model

# Create the DNN model
input_dim = X_train.shape[1]  # Number of features
num_classes = len(y.unique())  # Number of output classes ( S_B, S_M)

dnn_model = create_dnn_model(input_dim, num_classes)

# Train the DNN model
dnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model on the validation set
val_loss, val_accuracy = dnn_model.evaluate(X_val, y_val, verbose=0)
print(f'Validation Accuracy: {val_accuracy:.4f}')

# Make predictions on the validation set
y_val_pred = dnn_model.predict(X_val)
y_val_pred = y_val_pred.argmax(axis=1)  # Get the predicted class labels

# Evaluate the model
print("Classification Report (Validation):\n", classification_report(y_val, y_val_pred))

# Make predictions on the test set (optional)
y_test_pred = dnn_model.predict(X_test)
y_test_pred = y_test_pred.argmax(axis=1)

# Evaluate the model on the test set
test_loss, test_accuracy = dnn_model.evaluate(X_test, y_test, verbose=0)
print(f'Test Accuracy: {test_accuracy:.4f}')
print("Classification Report (Test):\n", classification_report(y_test, y_test_pred))

Epoch 1/10
4816/4816 [==============================] - 22s 4ms/step - loss: 0.1004 - accuracy: 0.9637 - val_loss: 0.0842 - val_accuracy: 0.9649
Epoch 2/10
4816/4816 [==============================] - 20s 4ms/step - loss: 0.0833 - accuracy: 0.9676 - val_loss: 0.0765 - val_accuracy: 0.9682
Epoch 3/10
4816/4816 [==============================] - 19s 4ms/step - loss: 0.0755 - accuracy: 0.9697 - val_loss: 0.0692 - val_accuracy: 0.9730
Epoch 4/10
4816/4816 [==============================] - 20s 4ms/step - loss: 0.0703 - accuracy: 0.9721 - val_loss: 0.0656 - val_accuracy: 0.9736
Epoch 5/10
4816/4816 [==============================] - 21s 4ms/step - loss: 0.0666 - accuracy: 0.9729 - val_loss: 0.0660 - val_accuracy: 0.9737
Epoch 6/10
4816/4816 [==============================] - 21s 4ms/step - loss: 0.0635 - accuracy: 0.9744 - val_loss: 0.0615 - val_accuracy: 0.9744
Epoch 7/10
4816/4816 [==============================] - 20s 4ms/step - loss: 0.0619 - accuracy: 0.9753 - val_loss: 0.0599 - val_ac

In [44]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
# Compute class weights based on the class distribution in the target variable y
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))

In [45]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [46]:
# Define the DNN model
model = Sequential([
    Dense(128, activation='relu', input_dim=X_train.shape[1]),  # Input layer
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),  # Hidden layer
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),  # Hidden layer
    BatchNormalization(),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Output layer (binary classification)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [47]:
# Define the DNN model
model = Sequential([
    Dense(128, activation='relu', input_dim=X_train.shape[1]),  # Input layer
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),  # Hidden layer
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),  # Hidden layer
    BatchNormalization(),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Output layer (binary classification)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [49]:
# Early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[early_stopping]
)


Epoch 1/10
4816/4816 [==============================] - 31s 6ms/step - loss: 0.4062 - accuracy: 0.7787 - val_loss: 0.3685 - val_accuracy: 0.8281
Epoch 2/10
4816/4816 [==============================] - 32s 7ms/step - loss: 0.3108 - accuracy: 0.8382 - val_loss: 0.3489 - val_accuracy: 0.8305
Epoch 3/10
4816/4816 [==============================] - 34s 7ms/step - loss: 0.2792 - accuracy: 0.8572 - val_loss: 0.3144 - val_accuracy: 0.8492
Epoch 4/10
4816/4816 [==============================] - 37s 8ms/step - loss: 0.2616 - accuracy: 0.8692 - val_loss: 0.3157 - val_accuracy: 0.8599
Epoch 5/10
4816/4816 [==============================] - 34s 7ms/step - loss: 0.2477 - accuracy: 0.8773 - val_loss: 0.2764 - val_accuracy: 0.8865
Epoch 6/10
4816/4816 [==============================] - 34s 7ms/step - loss: 0.2340 - accuracy: 0.8877 - val_loss: 0.2260 - val_accuracy: 0.9049
Epoch 7/10
4816/4816 [==============================] - 38s 8ms/step - loss: 0.2266 - accuracy: 0.8919 - val_loss: 0.2265 - val_ac

In [50]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Predict on test data
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Generate classification report and confusion matrix
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

1517/1517 [==============================] - 4s 3ms/step - loss: 0.2453 - accuracy: 0.8977
Test Loss: 0.2452559918165207
Test Accuracy: 0.8976598381996155
1517/1517 [==============================] - 4s 2ms/step
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.90      0.94     46727
           1       0.26      0.95      0.41      1817

    accuracy                           0.90     48544
   macro avg       0.63      0.92      0.68     48544
weighted avg       0.97      0.90      0.92     48544

Confusion Matrix:
 [[41846  4881]
 [   87  1730]]


In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
# Train the logistic regression model
logistic_model = LogisticRegression(random_state=42)
logistic_model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_train_pred = logistic_model.predict(X_train)
y_val_pred = logistic_model.predict(X_val)
y_test_pred = logistic_model.predict(X_test)

c:\Users\Kanza Nasim\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [57]:
print("Logistic Regression - Training Set:")
print(classification_report(y_train, y_train_pred))
print("Logistic Regression - Validation Set:")
print(classification_report(y_val, y_val_pred))
print("Logistic Regression - test Set:")
print(classification_report(y_test, y_test_pred))

Logistic Regression - Training Set:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98    148335
           1       0.76      0.22      0.35      5768

    accuracy                           0.97    154103
   macro avg       0.86      0.61      0.66    154103
weighted avg       0.96      0.97      0.96    154103

Logistic Regression - Validation Set:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98     36713
           1       0.73      0.22      0.33      1427

    accuracy                           0.97     38140
   macro avg       0.85      0.61      0.66     38140
weighted avg       0.96      0.97      0.96     38140

Logistic Regression - test Set:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98     46727
           1       0.70      0.20      0.31      1817

    accuracy                           0.97     48544
   macro avg       0.8